In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable as V
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
batch_size = 64

train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())

train_loader = DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [3]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x)

In [4]:
model = Net()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [5]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = V(data), V(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = V(data, volatile=True), V(target)
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).data[0]
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
for epoch in range(1, 10):
    train(epoch)
    test()

/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.313692
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.294715
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.273031
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.254467
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.240789
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.221239
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.189259
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.157744
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.106220
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.994917
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.768206
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.572825
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.417828
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.028137
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.943039
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.888157
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.649043
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.637816
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.586120
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.627477
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.226710
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.178125
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.066168
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.147341
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.195424
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.089145
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.084541
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.147480
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.184419
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.079873
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.215536
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.073825
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.313029
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.171699
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.064752
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.060602
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.028979
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.217529
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.060447
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.196589


Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.108684
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.090914
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.028892
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.101059
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.136515
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.116880
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.087875
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.117456
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.106956
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.086443
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.073420
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.136016
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.077310
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.107599
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.071841
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.075171
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.092409
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.045523
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.015336
Train Epoch: 4 [41600/60000 (69%)]	Loss: 0.046446


Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.061869
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.117032
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.027275
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.051891
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.036932
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.133743
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.076703
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.018684
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.053197
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.084879
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.132866
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.116296
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.063799
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.065860
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.100465
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.029870
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.035616
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.038140
Train Epoch: 6 [25600/60000 (43%)]	Loss: 0.019355
Train Epoch: 6 [26240/60000 (44%)]	Loss: 0.068531


Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.016191
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.059261

Test set: Average loss: 0.0705, Accuracy: 9778/10000 (98%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.066979
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.079321
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.062436
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.064810
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.025008
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.110655
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.071439
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.029783
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.050882
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.017249
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.010493
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.113879
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.031888
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.055724
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.076618
Train Epoch: 8 [9600/60000 (16%)]	Loss: 0.090308
Train Epoch: 8 [10240/60000 (17%)]	Loss: 0.047116
Train Epoch: 8 [10

Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.033079
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.187832
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.022270
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.035412
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.040377
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.047004
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.069468
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.044993
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.065818
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.009843
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.075741
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.022614
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.048448
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.023314
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.016506
Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.061420
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.012169
Train Epoch: 9 [54400/60000 (91%)]	Loss: 0.014863
Train Epoch: 9 [55040/60000 (92%)]	Loss: 0.076913
Train Epoch: 9 [55680/60000 (93%)]	Loss: 0.030460
